In [1]:
# reextract_one_embedding.py
# Re-extracts a single embedding safely in FP32

from pathlib import Path
from PIL import Image
import numpy as np
import torch
from torch import nn
import timm
from torchvision import transforms

# ---------------- CONFIG ----------------
ROOT = Path.cwd().parent
SPLIT = "train"
VIDEO_STEM = "837"   # <<< CHANGE IF NEEDED

FRAMES_ROOT = ROOT / "preprocessed" / "frames"
EMB_ROOT = ROOT / "embeddings"
CHECKPOINT_PATH = ROOT / "checkpoints" / "spatial" / "spatial_best_valAUC.pth"

IMG_SIZE = 224
BATCH = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# --------------------------------------

print("Device:", DEVICE)

# ---------------- MODEL ----------------
class SpatialModel(nn.Module):
    def __init__(self, backbone_name="efficientnet_b3", pretrained=False, head_hidden=512, dropout=0.4):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0)
        feat_dim = self.backbone.num_features
        self.head = nn.Sequential(
            nn.Linear(feat_dim, head_hidden),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(head_hidden, 1)
        )

    def forward(self, x):
        feats = self.backbone(x)
        logits = self.head(feats).squeeze(1)
        return logits

model = SpatialModel(pretrained=False).to(DEVICE)

ck = torch.load(CHECKPOINT_PATH, map_location="cpu")
state = ck.get("model_state", ck)

# strip "module." if present
new_state = {}
for k, v in state.items():
    nk = k.replace("module.", "") if k.startswith("module.") else k
    new_state[nk] = v

model.load_state_dict(new_state, strict=False)
model.eval()

print("Model loaded.")

# ---------------- PREPROCESS ----------------
preprocess = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

def load_frame(p):
    img = Image.open(p).convert("RGB")
    return preprocess(img)

# ---------------- RE-EXTRACT ----------------
frames_dir = FRAMES_ROOT / SPLIT / VIDEO_STEM
out_path = EMB_ROOT / SPLIT / f"{VIDEO_STEM}.npy"

if not frames_dir.exists():
    raise FileNotFoundError(f"Frames not found: {frames_dir}")

# delete old corrupted file
if out_path.exists():
    out_path.unlink()
    print("Deleted old embedding:", out_path)

frame_files = sorted(frames_dir.glob("frame_*.jpg"))
print("Frames found:", len(frame_files))

tensors = [load_frame(p) for p in frame_files]

embeddings = []

with torch.no_grad():
    for i in range(0, len(tensors), BATCH):
        batch = torch.stack(tensors[i:i+BATCH]).to(DEVICE)
        feats = model.backbone(batch)   # ❗ FP32 ONLY
        embeddings.append(feats.cpu().numpy())

embeddings = np.vstack(embeddings)
np.save(out_path, embeddings)

print("Saved new embedding:", out_path)
print("Shape:", embeddings.shape)

# ---------------- VERIFY ----------------
print("Verification:")
print("any NaN:", np.isnan(embeddings).any())
print("any Inf:", np.isinf(embeddings).any())
print("min/max/mean:",
      np.min(embeddings),
      np.max(embeddings),
      np.mean(embeddings))

Device: cuda
Model loaded.
Deleted old embedding: c:\Users\lkmah\OneDrive\Desktop\Lokesh\VS Code\DeepFake_Detection_SIC\embeddings\train\837.npy
Frames found: 8
Saved new embedding: c:\Users\lkmah\OneDrive\Desktop\Lokesh\VS Code\DeepFake_Detection_SIC\embeddings\train\837.npy
Shape: (8, 1536)
Verification:
any NaN: False
any Inf: False
min/max/mean: -0.23082626 1244.7639 7.593655
